In [1]:
import vectorbtpro as vbt

from vctr.data.data_loader import get_data, get_data_with_features_and_labels

# data = get_data('CRV', '1h', '2022-01-01')

# data = get_data_with_features_and_labels('MATIC', '1h', separate=False, label_args=(0.065, 0.005), start='2022-06-01 00:00:00')
data1 = get_data_with_features_and_labels(
    'CRV', '15m', separate=False, label_args=(0.03, 0.005), start='2022-01-01'
)
data2 = get_data_with_features_and_labels(
    'BTC', '15m', separate=False, label_args=(0.03, 0.005), start='2022-01-01'
)


In [3]:
len(data2.columns)

296

In [ ]:
from os import environ
from alpha_vantage.indicators import Indicators
import pandas as pd

ts = TimeSeries(key='8HHTNOWUT8NO6N6N', output_format='pandas')

# Add two new columns to the data frame for the macroeconomic indicators
data['indicator_1'] = None
data['indicator_2'] = None

# Iterate over each row in the data frame
for index, row in data.iterrows():
    # Retrieve the macroeconomic indicators from Alpha Vantage
    indicator_1, meta_data = ts.get_macroeconomic_indicators(symbol='REAL_GDP', interval='60min', outputsize='compact')
    indicator_2, meta_data = ts.get_macroeconomic_indicators(symbol='CPI', interval='60min', outputsize='compact')

    # Get the current row's date
    current_date = row['date']

    # Update the corresponding row with the retrieved indicators
    data.at[index, 'indicator_1'] = indicator_1[current_date][0]
    data.at[index, 'indicator_2'] = indicator_2[current_date][0]

# Print the data frame
print(data)


In [ ]:
import requests


def get_interest_rates():
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey=8HHTNOWUT8NO6N6N'
    r = requests.get(url)
    return r.json()


In [ ]:
get_interest_rates()['data']

In [ ]:
from vctr.features.feature_engineering import get_economic_indicator


gdp = get_economic_indicator('REAL_GDP')


In [ ]:
gdp

In [ ]:
import pandas as pd
import numpy as np


def add_interest_rate_column(dataframe, interest_rate_data):
    # Convert interest_rate_data to a DataFrame
    interest_rate_df = pd.DataFrame(interest_rate_data['data'])

    # Convert the 'date' column to datetime type
    interest_rate_df['date'] = pd.to_datetime(interest_rate_df['date'])

    # Convert the 'value' column to numeric type
    interest_rate_df['value'] = pd.to_numeric(interest_rate_df['value'])

    # Set the 'date' column as the index and convert to UTC timezone
    interest_rate_df.set_index('date', inplace=True)
    interest_rate_df.index = interest_rate_df.index.tz_localize('UTC')

    # Convert the dataframe's index to UTC timezone if it's tz-naive or has a different timezone
    if dataframe.index.tz is None or dataframe.index.tz.zone != 'UTC':
        dataframe = dataframe.tz_convert('UTC')

    # Determine the desired frequency from the dataframe
    frequency = dataframe.index.to_series().diff().mode().iloc[0]

    # Drop duplicates before resampling
    dataframe = dataframe.loc[~dataframe.index.duplicated(keep='first')]
    interest_rate_df = interest_rate_df.loc[~interest_rate_df.index.duplicated(keep='first')]

    # Resample and forward fill the interest rate data to match the desired frequency
    resampled_interest_rate = interest_rate_df.resample(frequency).last().ffill()

    # Merge the dataframes based on the indices
    merged_df = dataframe.merge(resampled_interest_rate, how='left', left_index=True, right_index=True)

    # Forward fill the remaining NaN values with the last non-NaN value
    merged_df['value'] = merged_df['value'].fillna(method='ffill')

    # Rename the 'value' column to 'interest_rate'
    merged_df.rename(columns={'value': 'interest_rate'}, inplace=True)

    return merged_df


In [ ]:
df = add_interest_rate_column(data, get_interest_rates())

In [ ]:
interest_rates = get_interest_rates()

In [ ]:
interest_rates

In [ ]:
df

In [ ]:
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM

def add_hmm_features(df):
  # Create an HMM model.
  hmm = GaussianHMM(n_components=2, covariance_type='full')

  # Train the HMM model on the OHLCV data.
  hmm.fit(df[['close']])

  # Generate a sequence of hidden states for the future.
  hidden_states = hmm.predict(df[['close']])

  # Add the hidden states to the dataframe.
  df['hmm_hidden'] = hidden_states

  return df


In [ ]:
df = add_hmm_features(data)

In [ ]:
df['close'].vbt.overlay_with_heatmap(df['hmm_hidden']).show()

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# instantiate the variance threshold selector with the threshold value
selector = VarianceThreshold(threshold=0.1)

# fit the selector to the data
selector.fit(data)

# obtain the boolean mask of the selected features
buy_mask = selector.get_support()

# get the names of the selected features
selected_features = [col for col, m in zip(data.columns, buy_mask) if m]
print("==>> selected_features: ", selected_features)

# remove the redundant features from the dataframe
X_train = data[selected_features]

# X_train.columns

In [ ]:
import requests


def get_economic_indicator(indicator):
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function={indicator}&interval=daily&apikey=8HHTNOWUT8NO6N6N'
    r = requests.get(url)
    return r.json()['data']


In [ ]:
for ind in ['FEDERAL_FUNDS_RATE', 'REAL_GDP', 'CPI', 'INFLATION', 'RETAIL_SALES', 'DURABLES', 'UNEMPLOYMENT', 'TREASURY_YIELD']:
    print(get_economic_indicator(ind))

In [ ]:
from vctr.utils.workflow import run_workflow
from vctr.data import vbt
from vctr.features.feature_engineering import add_features
from vctr.data.labeling import label_data_extrema_multi
from vctr.data.data_loader import get_data, get_data_with_features_and_labels

df = get_data_with_features_and_labels(
    'ZEC', '1h', separate=False, label_args=(0.065, 0.005), crypto=True
)
# data2 = get_data_with_features_and_labels(
#     'MANA', '1h', separate=False, label_args=(0.0324, 0.0077), start='2023-01-28 00:00:00'
# )

# data = get_data('ATOM', '1h')
# data = add_features(data)


In [ ]:
print(len(df.columns))

In [ ]:
print(len(df.columns))
# print(len(data2.columns))

# print(list(set(data2.columns) - set(data.columns)))

In [ ]:
import pandas as pd
# Filter highly correlated features (pearson correlation > 0.8)
corr = df.corr()

correlated_features = set()
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.85:
            colname = corr.columns[i]
            correlated_features.add(colname)

# Print the list of correlated features
print(list(correlated_features))


In [ ]:
import vectorbtpro as vbt
from vctr.data.data_loader import get_data, get_data_with_features_and_labels

_data = get_data_with_features_and_labels('MATIC', '1h', separate=False, label_args=(0.04, 0.0075), start='2022-01-01 00:00:00', no_cache=True)

# Slice the data so that we cut off everything after 03-11
# _data = _data.loc[:'2023-03-10 10:00:00']


In [ ]:
len(_data.columns)

In [ ]:
from vctr.data import vbt

df = df.copy()['2023':]

fig = df.vc.plot(plot_volume=False)
bbands = df.vc.run('bbands')
rsi = df.vc.run('rsi')
sd_bb = vbt.SIGDET.run(bbands.bandwidth, factor=2)
# sd_bb.signal.vbt.plot(fig=fig)

# Establish masks for plotting
sigdet_up = sd_bb.signal == 1
sigdet_dn = sd_bb.signal == -1

# buy_mask = data.label == 1
# buy_mask.vbt.signals.ranges.plot_shapes(plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='green'))
# sell_mask = data.label == 2
# sell_mask.vbt.signals.ranges.plot_shapes(plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='red'))

piv = vbt.PIVOTINFO.run(df.high, df.low, 0.25, 0.15)
# Buy?
piv_buy_mask = piv.last_value_crossed_below(df.low)
piv_buy_mask.vbt.signals.ranges.plot_shapes(
    plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='green', opacity=1)
)
piv_sell_mask = piv.last_value_crossed_above(bbands.upper)

piv_sell_mask.vbt.signals.ranges.plot_shapes(
    plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='orange', opacity=1)
)

piv.conf_value_crossed_above(df.close).vbt.signals.ranges.plot_shapes(
    plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='red', opacity=1)
)

# print(piv.modes)
# print(piv.pivots)

fig.update_layout(width=1200, height=650)
fig.show()


In [ ]:
fig = d2.vc.plot(plot_volume=False, width=1200, height=650)

# rsi = d2.vc.run('rsi')
# sig = vbt.SIGDET.run(rsi.rsi, factor=4)
# up, dn = sig.signal == 1, sig.signal == -1

macd = vbt.talib('MACD').run(d2.close, fastperiod=14, slowperiod=32, signalperiod=9)
sig2 = vbt.SIGDET.run(macd.macd, factor=4)
up2, dn2 = sig2.signal == 1, sig2.signal == -1


up2.vbt.signals.ranges.plot_shapes(plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='cyan', opacity=.3))
dn2.vbt.signals.ranges.plot_shapes(plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='yellow', opacity=.3))

fig.show()

In [ ]:
fig = sigdet_up.vbt.signals.ranges.plot_shapes(
    plot_ohlc=df,
    width=1200,
    height=600,
    ohlc_type='candlestick',
    shape_kwargs=dict(fillcolor='green', opacity=0.25),
)
sigdet_dn.vbt.signals.ranges.plot_shapes(
    plot_ohlc=df,
    width=1200,
    height=600,
    ohlc_type='candlestick',
    shape_kwargs=dict(fillcolor='red', opacity=0.25),
    fig=fig,
).show()


In [ ]:
from vctr.data.data_loader import get_data, get_data_with_features_and_labels

df = get_data_with_features_and_labels('MATIC', '1h', separate=False, label_args=(0.04, 0.0075), start='2023-01-01 00:00:00')

In [ ]:
import pandas as pd
import numpy as np
import talib as ta


def add_supertrend(df, atr_period=10, atr_multiplier=2.0, change_atr=True):
    hl2 = (df['high'] + df['low']) / 2
    atr = (
        ta.ATR(df['high'], df['low'], df['close'], timeperiod=atr_period)
        if change_atr
        else ta.TRANGE(df['high'], df['low'], df['close']).rolling(window=atr_period).mean()
    )

    up = hl2 - (atr_multiplier * atr)
    dn = hl2 + (atr_multiplier * atr)

    up1 = up.shift(1).fillna(up)
    dn1 = dn.shift(1).fillna(dn)

    df['st_up'] = np.where(df['close'].shift(1) > up1, np.maximum(up, up1), up)
    df['st_dn'] = np.where(df['close'].shift(1) < dn1, np.minimum(dn, dn1), dn)

    df['st_trend'] = np.where(
        df['close'] > df['st_dn'].shift(1), 1, np.where(df['close'] < df['st_up'].shift(1), -1, np.nan)
    )
    df['st_trend'].fillna(method='ffill', inplace=True)

    df['st_buy'] = (df['st_trend'] == 1) & (df['st_trend'].shift(1) == -1).astype(int)
    df['st_sell'] = (df['st_trend'] == -1) & (df['st_trend'].shift(1) == 1).astype(int)

    df.drop(columns=['st_up', 'st_dn', 'st_trend'], inplace=True)

    return df


In [ ]:
d2 = df.copy()

d2 = add_supertrend(d2, 10, 3.0)

In [ ]:

from plotly.graph_objs import candlestick
import vectorbtpro as vbt

d2 = df.copy()

d2 = add_supertrend(d2, 10, 2)
fig = d2.vbt.ohlcv.plot(width=1200, height=650, plot_volume=False)
one, two = d2.st_buy == 1, d2.st_sell == 1
one.vbt.signals.ranges.plot_shapes(plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='green', opacity=0.7))
two.vbt.signals.ranges.plot_shapes(plot_close=False, fig=fig, shape_kwargs=dict(fillcolor='red', opacity=0.7))

fig.show()


In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=TMP0580D0ZSFILSH'
r = requests.get(url)
data = r.json()

print(data)
